<a href="https://colab.research.google.com/github/athishr88/NN_DL/blob/main/Coding_tutorial6/CT6_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# LSTM

#1. Load sample text file
import tensorflow as tf
path_to_file = tf.keras.utils.get_file("nietzsche.txt", 
                  origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")

614400/600901 [==============================] - 0s 1us/step


In [ ]:
#2. Read the text
text = open(path_to_file, "rb").read().decode(encoding='utf-8')
text = text.lower()

# Number of characters in the text
n_char = len(text)
print(f"Length of text: {n_char} characters")

Length of text: 600893 characters


In [ ]:
#3. List of unique characters

vocab = sorted(set(text))
num_vocab = len(vocab)
num_vocab

57

In [ ]:
#4. Create a mapping for each text
char_to_int = dict((c,i) for i, c in enumerate(vocab))

int_to_char = dict((i,c) for i, c in enumerate(vocab))

In [ ]:
# 5. Prepare the dataset of input to output pairs
## "Hello" --> input: "Hell" output: "o"

seq_length = 40
dataX = []
dataY = []

for i in range(0, n_char-seq_length, 1):
    seq_in = text[i:i+seq_length] # Inout substring
    seq_out = text[i+seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    
n_patterns = len(dataX)
print(f"Total patterns :{n_patterns}")
len(dataX)

Total patterns :600853


600853

In [ ]:
len(dataX)

600853

In [ ]:
#6. Preprocessing the input data
from tensorflow.keras.utils import to_categorical
import numpy as np

# Reshape X to be [Samples, time_steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# Normalize
X = X/float(num_vocab)

# One hot encode
y = to_categorical(dataY)



In [ ]:
X.shape

(600853, 40, 1)

In [ ]:
# 7. Build the model

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
#8. Train and save model

filepath = "LSTM_tutorial_model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
history = model.fit(X, y, epochs=20, batch_size=128, callbacks=[checkpoint])

In [ ]:
#9. Loading weights
model.load_weights(filepath)
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
#10. Predict characters using model
import sys
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Inpput:")
print("\"", "".join([int_to_char[value] for value in pattern]), "\"")
print("Predicted: ")
print("----------")

# Generate characters
for i in range(100):
  x = np.reshape(pattern, (1, len(pattern), 1))
  x = x/float(num_vocab)
  prediction = model.predict(x, verbose=0)
  index = np.argmax(prediction)
  result = int_to_char[index]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

print("\n-----------------")